### Import Required Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# -------------------------------------------------------------------------------------
from scipy.stats import chi2_contingency
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy.stats import f_oneway
# -------------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score,precision_recall_fscore_support,classification_report,r2_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier

import warnings
import os
import time

In [2]:
pd.options.display.max_columns=None
pd.options.display.max_rows=None

### Reading Data

In [3]:
a=pd.read_excel('case_study1.xlsx')
b=pd.read_excel('case_study2.xlsx')

### Create A Copy Of Your Data

In [4]:
df1=a.copy()
df2=b.copy()

### Dimensions Of Data

In [5]:
df1.shape

(51336, 26)

In [6]:
df2.shape

(51336, 62)

### Checking Null Values In df1

In [7]:
dim=[]
for i in df1.columns:
    dim.append((df1[df1[i].values==-99999].shape[0]))

pd.DataFrame(dim,index=df1.columns,columns=['Null Count'])

,Null Count
PROSPECTID,0
Total_TL,0
Tot_Closed_TL,0
Tot_Active_TL,0
Total_TL_opened_L6M,0
Tot_TL_closed_L6M,0
pct_tl_open_L6M,0
pct_tl_closed_L6M,0
pct_active_tl,0
pct_closed_tl,0


### Removing Null values

In [8]:
df1=df1[df1['Age_Oldest_TL']!= -99999]

In [9]:
df1.shape

(51296, 26)

### Checking Null Values In df2

In [10]:
dim=[]
for i in df2.columns:
    dim.append((df2[df2[i].values==-99999].shape[0]))

pd.DataFrame(dim,index=df2.columns,columns=['Null Count'])

,Null Count
PROSPECTID,0
time_since_recent_payment,4291
time_since_first_deliquency,35949
time_since_recent_deliquency,35949
num_times_delinquent,0
max_delinquency_level,35949
max_recent_level_of_deliq,0
num_deliq_6mts,0
num_deliq_12mts,0
num_deliq_6_12mts,0


### Removing Null Values In df2
- In df2, there are many columns have -99999 value i.e. null values.
- so, check the count of -99999 for every column and perform the following operation
- if count greater than 10000 then drop that column
- else count less than 10000 then drop that particular row

### Columns Removed

In [11]:
cols_to_be_removed=[]

for i in df2.columns:
    if df2[df2[i] == -99999].shape[0] > 10000:
        cols_to_be_removed.append(i)

In [12]:
cols_to_be_removed

['time_since_first_deliquency',
 'time_since_recent_deliquency',
 'max_delinquency_level',
 'max_deliq_6mts',
 'max_deliq_12mts',
 'CC_utilization',
 'PL_utilization',
 'max_unsec_exposure_inPct']

In [13]:
df2=df2.drop(cols_to_be_removed,axis=1)

In [14]:
df2.shape

(51336, 54)

### Rows Removed

In [15]:
for i in df2.columns:
    df2=df2[df2[i] != -99999]

In [16]:
df2.shape

(42066, 54)

### Checking Null Values

In [17]:
dim=[]
for i in df1.columns:
    dim.append((df1[df1[i].values==-99999].shape[0]))

pd.DataFrame(dim,index=df1.columns,columns=['Null Count'])

,Null Count
PROSPECTID,0
Total_TL,0
Tot_Closed_TL,0
Tot_Active_TL,0
Total_TL_opened_L6M,0
Tot_TL_closed_L6M,0
pct_tl_open_L6M,0
pct_tl_closed_L6M,0
pct_active_tl,0
pct_closed_tl,0


In [18]:
dim=[]
for i in df2.columns:
    dim.append((df2[df2[i].values==-99999].shape[0]))

pd.DataFrame(dim,index=df2.columns,columns=['Null Count'])

,Null Count
PROSPECTID,0
time_since_recent_payment,0
num_times_delinquent,0
max_recent_level_of_deliq,0
num_deliq_6mts,0
num_deliq_12mts,0
num_deliq_6_12mts,0
num_times_30p_dpd,0
num_times_60p_dpd,0
num_std,0


### Checking Common Columns Name

In [19]:
for i in df1.columns:
    if i in df2.columns:
        print(i)

PROSPECTID


### Merge df1 & df2 
- Perform inner join so that no null values are present

In [20]:
df=df1.merge(df2,how='inner',on='PROSPECTID')

In [21]:
df.shape

(42064, 79)

In [22]:
df.head()

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,Total_TL_opened_L12M,Tot_TL_closed_L12M,pct_tl_open_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,Auto_TL,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,num_times_delinquent,max_recent_level_of_deliq,num_deliq_6mts,num_deliq_12mts,num_deliq_6_12mts,num_times_30p_dpd,num_times_60p_dpd,num_std,num_std_6mts,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_6mts,num_dbt_12mts,num_lss,num_lss_6mts,num_lss_12mts,recent_level_of_deliq,tot_enq,CC_enq,CC_enq_L6m,CC_enq_L12m,PL_enq,PL_enq_L6m,PL_enq_L12m,time_since_recent_enq,enq_L12m,enq_L6m,enq_L3m,MARITALSTATUS,EDUCATION,AGE,GENDER,NETMONTHLYINCOME,Time_With_Curr_Empr,pct_of_active_TLs_ever,pct_opened_TLs_L6m_of_L12m,pct_currentBal_all_TL,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,0,0,0.00,0.000,0,0,0,0,1,0,4,1,4,0,72,18,549,11,29,0,0,0,0,0,21,5,11,0,0,0,0,0,0,0,0,0,29,6,0,0,0,6,0,0,566,0,0,0,Married,12TH,48,M,51000,114,0.200,0.0,0.798,0,1,0.0,0.0,0.000,0.0,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,1,0,1.00,0.000,0,0,0,1,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,209,1,0,0,Single,GRADUATE,23,F,19000,50,1.000,0.0,0.370,0,0,0.0,0.0,0.000,0.0,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,2,0,0.25,0.000,1,1,0,6,1,0,0,2,6,0,47,2,302,9,25,1,9,8,0,0,10,5,10,0,0,0,0,0,0,0,0,0,25,4,0,0,0,0,0,0,587,0,0,0,Married,SSC,40,M,18,191,1.000,0.5,0.585,0,0,0.0,0.0,0.000,0.0,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,0,0,0.00,0.000,0,1,0,0,0,0,0,3,0,2,131,32,583,0,0,0,0,0,0,0,53,4,16,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,3951,0,0,0,Married,POST-GRADUATE,48,M,15000,75,0.333,0.0,0.000,0,0,0.0,0.0,0.000,0.0,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,0,1,0.00,0.167,0,4,0,0,2,0,0,6,0,0,150,17,245,14,270,0,0,0,13,11,5,0,2,3,0,1,0,0,0,0,0,0,26,15,2,0,1,7,3,3,7,6,5,4,Married,12TH,35,M,0,154,0.167,0.0,0.000,0,0,1.0,0.0,0.429,0.0,1,0,ConsumerLoan,PL,668,P3


In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 79 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   PROSPECTID                  42064 non-null  int64  
 1   Total_TL                    42064 non-null  int64  
 2   Tot_Closed_TL               42064 non-null  int64  
 3   Tot_Active_TL               42064 non-null  int64  
 4   Total_TL_opened_L6M         42064 non-null  int64  
 5   Tot_TL_closed_L6M           42064 non-null  int64  
 6   pct_tl_open_L6M             42064 non-null  float64
 7   pct_tl_closed_L6M           42064 non-null  float64
 8   pct_active_tl               42064 non-null  float64
 9   pct_closed_tl               42064 non-null  float64
 10  Total_TL_opened_L12M        42064 non-null  int64  
 11  Tot_TL_closed_L12M          42064 non-null  int64  
 12  pct_tl_open_L12M            42064 non-null  float64
 13  pct_tl_closed_L12M          420

In [24]:
df.isna().sum().sum()

0

### We Will Divide The Features Into - 
- Categorical
- Numerical

### Check How Many Columns Are Categorical

In [25]:
cat_cols=[]
for i in df.columns:
    if df[i].dtype == 'object':
        cat_cols.append(i)

cat_cols

['MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

### Check Unique Value In Each Category

In [26]:
for i in cat_cols:
    print(i,' : ',df[i].unique())
    print()

MARITALSTATUS  :  ['Married' 'Single']

EDUCATION  :  ['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']

GENDER  :  ['M' 'F']

last_prod_enq2  :  ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']

first_prod_enq2  :  ['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']

Approved_Flag  :  ['P2' 'P1' 'P3' 'P4']



### For Handling Categorical Vs Categorical Data Use Chi-Square Test

- since all the categorical features have (pval<=0.05), we will accept all

In [27]:
for i in cat_cols[:-1]:
    chi2,pval,_,_=chi2_contingency(pd.crosstab(df[i],df['Approved_Flag']))
    print(i,' --- ',pval)

MARITALSTATUS  ---  3.578180861038862e-233
EDUCATION  ---  2.6942265249737532e-30
GENDER  ---  1.907936100186563e-05
last_prod_enq2  ---  0.0
first_prod_enq2  ---  7.84997610555419e-287


### Check How Many Columns Are Numerical

In [28]:
num_cols=[]

for i in df.columns:
    if df[i].dtype != 'object' and i not in ['PROSPECTID']:
        num_cols.append(i)

len(num_cols)

72

### Check Multicollinearity Between Numerical Columns Sequential Method

In [29]:
vif_data=df[num_cols]
total_columns = vif_data.shape[1]
cols_to_be_kept=[]
columns_index=0

for i in range(total_columns):
    vif_val=variance_inflation_factor(vif_data,columns_index)
    print(columns_index,' --- ',vif_val)

    if vif_val < 6:
        cols_to_be_kept.append(num_cols[i])
        columns_index += 1
    else:
        vif_data=vif_data.drop([num_cols[i]],axis=1)

C:\Users\DELL\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0  ---  inf


C:\Users\DELL\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


0  ---  inf
0  ---  11.320180023967996
0  ---  8.363698035000336
0  ---  6.520647877790928
0  ---  5.149501618212625
1  ---  2.611111040579735


C:\Users\DELL\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


2  ---  inf
2  ---  1788.7926256209232
2  ---  8.601028256477228
2  ---  3.8328007921530785
3  ---  6.099653381646723
3  ---  5.581352009642762
4  ---  1.9855843530987785


C:\Users\DELL\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


5  ---  inf
5  ---  4.809538302819343
6  ---  23.270628983464636
6  ---  30.595522588100053
6  ---  4.384346405965583
7  ---  3.0646584155234238
8  ---  2.898639771299253
9  ---  4.377876915347324
10  ---  2.207853583695844
11  ---  4.916914200506864
12  ---  5.214702030064725
13  ---  3.3861625024231476
14  ---  7.840583309478997
14  ---  5.255034641721434


C:\Users\DELL\anaconda3\Lib\site-packages\statsmodels\stats\outliers_influence.py:198: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


15  ---  inf
15  ---  7.380634506427232
15  ---  1.421005001517573
16  ---  8.083255010190323
16  ---  1.624122752404011
17  ---  7.257811920140003
17  ---  15.59624383268298
17  ---  1.825857047132431
18  ---  1.5080839450032666
19  ---  2.172088834824578
20  ---  2.6233975535272283
21  ---  2.2959970812106176
22  ---  7.360578319196446
22  ---  2.1602387773102554
23  ---  2.8686288267891475
24  ---  6.458218003637277
24  ---  2.8474118865638256
25  ---  4.7531981562840855
26  ---  16.227354755948223
26  ---  6.424377256363877
26  ---  8.887080381808687
26  ---  2.3804746142952666
27  ---  8.609513476514548
27  ---  13.06755093547673
27  ---  3.500040056654654
28  ---  1.908795587481377
29  ---  17.006562234161628
29  ---  10.730485153719197
29  ---  2.3538497522950275
30  ---  22.10485591513649
30  ---  2.7971639638512906
31  ---  3.424171203217696
32  ---  10.175021454450922
32  ---  6.408710354561292
32  ---  1.001151196262562
33  ---  3.069197305397273
34  ---  2.8091261600643724


In [30]:
len(cols_to_be_kept)

39

### Apply ANOVA Test For cols_to_be_kept

In [31]:
cols_to_be_kept_num=[]

for i in cols_to_be_kept:
    a=list(df[i])
    b=list(df['Approved_Flag'])

    group_p1=[value for value,group in zip(a,b) if group == 'P1']
    group_p2=[value for value,group in zip(a,b) if group == 'P2']
    group_p3=[value for value,group in zip(a,b) if group == 'P3']
    group_p4=[value for value,group in zip(a,b) if group == 'P4']

    f_stat,p_val=f_oneway(group_p1,group_p2,group_p3,group_p4)

    if p_val <= 0.05:
        cols_to_be_kept_num.append(i)

In [32]:
len(cols_to_be_kept_num)

37

#### Feature Selection is done for categorical and numerical features

In [33]:
df=df[list(cols_to_be_kept_num + cat_cols)]

In [34]:
df.shape

(42064, 43)

In [35]:
df.head()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,MARITALSTATUS,EDUCATION,GENDER,last_prod_enq2,first_prod_enq2,Approved_Flag
0,0.000,0.0,0,0.000,0,0,0,4,1,4,0,72,18,549,29,0,0,11,0,0,0,0,0,0,29,0,0,566,0,51000,114,0,1,0.000,0.0,1,0,Married,12TH,M,PL,PL,P2
1,0.000,0.0,0,0.000,0,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,0,0,209,0,19000,50,0,0,0.000,0.0,0,0,Single,GRADUATE,F,ConsumerLoan,ConsumerLoan,P2
2,0.125,0.0,0,0.000,1,0,0,0,2,6,0,47,2,302,25,8,0,10,0,0,0,0,0,0,25,0,0,587,0,18,191,0,0,0.000,0.0,1,0,Married,SSC,M,ConsumerLoan,others,P2
3,0.000,0.0,0,0.000,0,0,0,0,3,0,2,131,32,583,0,0,0,16,0,0,0,0,0,0,0,0,0,3951,0,15000,75,0,0,0.000,0.0,0,0,Married,POST-GRADUATE,M,AL,AL,P1
4,0.000,0.0,1,0.167,0,0,0,0,6,0,0,150,17,245,270,0,11,2,3,0,1,0,0,0,26,1,3,7,4,0,154,0,0,0.429,0.0,1,0,Married,12TH,M,ConsumerLoan,PL,P3


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42064 entries, 0 to 42063
Data columns (total 43 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            42064 non-null  float64
 1   pct_tl_closed_L6M          42064 non-null  float64
 2   Tot_TL_closed_L12M         42064 non-null  int64  
 3   pct_tl_closed_L12M         42064 non-null  float64
 4   Tot_Missed_Pmnt            42064 non-null  int64  
 5   CC_TL                      42064 non-null  int64  
 6   Home_TL                    42064 non-null  int64  
 7   PL_TL                      42064 non-null  int64  
 8   Secured_TL                 42064 non-null  int64  
 9   Unsecured_TL               42064 non-null  int64  
 10  Other_TL                   42064 non-null  int64  
 11  Age_Oldest_TL              42064 non-null  int64  
 12  Age_Newest_TL              42064 non-null  int64  
 13  time_since_recent_payment  42064 non-null  int

In [37]:
df.duplicated().sum()

2

In [38]:
df.drop_duplicates(keep='first',inplace=True)

In [39]:
df.duplicated().sum()

0

In [40]:
df.shape

(42062, 43)

### Label Encoding For The Categorical Features

In [41]:
cat_cols=['MARITALSTATUS','EDUCATION', 'GENDER', 'last_prod_enq2', 'first_prod_enq2']

for i in cat_cols:
    print(i,' : ',df[i].unique())
    print()

MARITALSTATUS  :  ['Married' 'Single']

EDUCATION  :  ['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS'
 'PROFESSIONAL']

GENDER  :  ['M' 'F']

last_prod_enq2  :  ['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']

first_prod_enq2  :  ['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']



#### Ordinal feature --- education
- ssc :            1
- 12th :           2
- graduate :       3
- under graduate : 3
- post graduate :  4
- others :         1
- professional :   3

In [42]:
df.loc[df['EDUCATION']=='SSC',['EDUCATION']]=1
df.loc[df['EDUCATION']=='12TH',['EDUCATION']]=2
df.loc[df['EDUCATION']=='GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='UNDER GRADUATE',['EDUCATION']]=3
df.loc[df['EDUCATION']=='POST-GRADUATE',['EDUCATION']]=4
df.loc[df['EDUCATION']=='OTHERS',['EDUCATION']]=1
df.loc[df['EDUCATION']=='PROFESSIONAL',['EDUCATION']]=3

In [43]:
df['EDUCATION'].value_counts()

EDUCATION
3    18930
2    11703
1     9531
4     1898
Name: count, dtype: int64

In [44]:
df['EDUCATION']=df['EDUCATION'].astype(int)

### One Hot Encoding

In [45]:
df_encoded=pd.get_dummies(df,columns=['MARITALSTATUS','GENDER', 'last_prod_enq2', 'first_prod_enq2'],dtype='int')

In [46]:
df_encoded.head()

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,EDUCATION,Approved_Flag,MARITALSTATUS_Married,MARITALSTATUS_Single,GENDER_F,GENDER_M,last_prod_enq2_AL,last_prod_enq2_CC,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
0,0.000,0.0,0,0.000,0,0,0,4,1,4,0,72,18,549,29,0,0,11,0,0,0,0,0,0,29,0,0,566,0,51000,114,0,1,0.000,0.0,1,0,2,P2,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0
1,0.000,0.0,0,0.000,0,0,0,0,0,1,0,7,7,47,0,0,0,0,0,0,0,0,0,0,0,0,0,209,0,19000,50,0,0,0.000,0.0,0,0,3,P2,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,0
2,0.125,0.0,0,0.000,1,0,0,0,2,6,0,47,2,302,25,8,0,10,0,0,0,0,0,0,25,0,0,587,0,18,191,0,0,0.000,0.0,1,0,1,P2,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1
3,0.000,0.0,0,0.000,0,0,0,0,3,0,2,131,32,583,0,0,0,16,0,0,0,0,0,0,0,0,0,3951,0,15000,75,0,0,0.000,0.0,0,0,4,P1,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0
4,0.000,0.0,1,0.167,0,0,0,0,6,0,0,150,17,245,270,0,11,2,3,0,1,0,0,0,26,1,3,7,4,0,154,0,0,0.429,0.0,1,0,2,P3,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0


In [47]:
df_encoded.shape 

(42062, 55)

In [48]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42062 entries, 0 to 42063
Data columns (total 55 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               42062 non-null  float64
 1   pct_tl_closed_L6M             42062 non-null  float64
 2   Tot_TL_closed_L12M            42062 non-null  int64  
 3   pct_tl_closed_L12M            42062 non-null  float64
 4   Tot_Missed_Pmnt               42062 non-null  int64  
 5   CC_TL                         42062 non-null  int64  
 6   Home_TL                       42062 non-null  int64  
 7   PL_TL                         42062 non-null  int64  
 8   Secured_TL                    42062 non-null  int64  
 9   Unsecured_TL                  42062 non-null  int64  
 10  Other_TL                      42062 non-null  int64  
 11  Age_Oldest_TL                 42062 non-null  int64  
 12  Age_Newest_TL                 42062 non-null  int64  
 13  time_s

## Machine Learning Model Fitting
### 1) Random Forest

In [49]:
X=df_encoded.drop('Approved_Flag',axis=1)
y=df_encoded['Approved_Flag']

In [50]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [51]:
rf=RandomForestClassifier(n_estimators=200,random_state=42)

In [52]:
rf.fit(X_train,y_train)

RandomForestClassifier(n_estimators=200, random_state=42)

In [53]:
y_pred=rf.predict(X_test)

In [54]:
accuracy=accuracy_score(y_test,y_pred)
print()
print(f'Accuracy: {accuracy}')
print()
precision , recall , f1_score , _ = precision_recall_fscore_support(y_test,y_pred)

for i,v in enumerate(['p1','p2','p3','p4']):
    print(f'class {v}:')
    print(f'precision {precision[i]}')
    print(f'recall {recall[i]}')
    print(f'f1_score {f1_score[i]}')
    print()


Accuracy: 0.7634613098775704

class p1:
precision 0.82063305978898
recall 0.7014028056112225
f1_score 0.7563479200432199

class p2:
precision 0.7939979654120041
recall 0.9306438791732909
f1_score 0.8569075937785909

class p3:
precision 0.46266233766233766
recall 0.21189591078066913
f1_score 0.29066802651708307

class p4:
precision 0.7217973231357553
recall 0.7273603082851637
f1_score 0.7245681381957773



### 2) XGBoost

In [89]:
X=df_encoded.drop('Approved_Flag',axis=1)
y=df_encoded['Approved_Flag']

In [90]:
encoder=LabelEncoder()

y=encoder.fit_transform(y)

In [91]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [92]:
xgb=XGBClassifier(objective='multi:softmax',num_class=4)

In [93]:
xgb.fit(X_train,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None, num_class=4,
              num_parallel_tree=None, ...)

In [94]:
y_pred=xgb.predict(X_test)

In [95]:
accuracy=accuracy_score(y_test,y_pred)
print()
print(f'Accuracy: {accuracy}')
print()
precision , recall , f1_score , _ = precision_recall_fscore_support(y_test,y_pred)

for i,v in enumerate(['p1','p2','p3','p4']):
    print(f'class {v}:')
    print(f'precision {precision[i]}')
    print(f'recall {recall[i]}')
    print(f'f1_score {f1_score[i]}')
    print()


Accuracy: 0.7704742660168786

class p1:
precision 0.8079331941544885
recall 0.7755511022044088
f1_score 0.7914110429447853

class p2:
precision 0.8167024320457796
recall 0.9075914149443561
f1_score 0.8597515060240964

class p3:
precision 0.45368171021377673
recall 0.28401486988847585
f1_score 0.34933699131229995

class p4:
precision 0.7433888344760039
recall 0.7312138728323699
f1_score 0.7372510927634773



### 3) DecisionTree

In [62]:
X=df_encoded.drop('Approved_Flag',axis=1)
y=df_encoded['Approved_Flag']

In [63]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [64]:
dt=DecisionTreeClassifier(max_depth=20,min_samples_split=10)

In [65]:
dt.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=20, min_samples_split=10)

In [66]:
y_pred=dt.predict(X_test)

In [67]:
accuracy=accuracy_score(y_test,y_pred)
print()
print(f'Accuracy: {accuracy}')
print()
precision , recall , f1_score , _ = precision_recall_fscore_support(y_test,y_pred)

for i,v in enumerate(['p1','p2','p3','p4']):
    print(f'class {v}:')
    print(f'precision {precision[i]}')
    print(f'recall {recall[i]}')
    print(f'f1_score {f1_score[i]}')
    print()


Accuracy: 0.7064067514560799

class p1:
precision 0.7023809523809523
recall 0.7094188376753507
f1_score 0.7058823529411765

class p2:
precision 0.7979583975346687
recall 0.8233306836248012
f1_score 0.8104460093896714

class p3:
precision 0.3468543046357616
recall 0.3115241635687732
f1_score 0.3282412847630239

class p4:
precision 0.6696517412935323
recall 0.6483622350674374
f1_score 0.6588350465002448



### ---------------------------------------------------------------------------------------------

In [68]:
df_encoded['Approved_Flag'].value_counts()

Approved_Flag
P2    25450
P3     6440
P4     5264
P1     4908
Name: count, dtype: int64

### Hyperparameter Tuning XgBoost

In [122]:
param_grid={
    'colsample_bytree':[0.1,0.3,0.5,0.7,0.9],
    'learning_rate':[0.001,0.01,0.1,1],
    'max_depth':[3,5,8,10],
    'alpha':[1,10,100],
    'n_estimators':[10,50,100]
}

In [ ]:
index=0

answers_grid={
    'combination':[],
    'train_Accuracy':[],
    'test_Accuracy':[],
    'colsample_bytree':[],
    'learning_rate':[],
    'max_depth':[],
    'alpha':[],
    'n_estimators':[]
}

# loop through each combination of hyperparameters

for colsample_bytree in param_grid['colsample_bytree']:
    for learning_rate in param_grid['learning_rate']:
        for max_depth in param_grid['max_depth']:
            for alpha in param_grid['alpha']:
                for n_estimators in param_grid['n_estimators']:
                    index = index + 1

                    model =XGBClassifier(objective='multi:softmax',
                                         num_class=4,
                                         colsample_bytree=colsample_bytree,
                                         learning_rate=learning_rate,
                                         alpha=alpha,
                                         max_depth=max_depth,
                                         n_estimators=n_estimators
                                        )
                    X=df_encoded.drop('Approved_Flag',axis=1)
                    y=df_encoded['Approved_Flag']

                    label_encoder=LabelEncoder()
                    y_encoded=label_encoder.fit_transform(y)

                    X_train,X_test,y_train,y_test=train_test_split(X,y_encoded,test_size=0.2,random_state=42)

                    model.fit(X_train,y_train)

                    y_pred_train=model.predict(X_train)
                    y_pred_test=model.predict(X_test)

                    train_accuracy=accuracy_score(y_train,y_pred_train)
                    test_accuracy=accuracy_score(y_test,y_pred_test)


                    answers_grid['combination'].append(index)
                    answers_grid['train_Accuracy'].append(train_accuracy)
                    answers_grid['test_Accuracy'].append(test_accuracy)
                    answers_grid['colsample_bytree'].append(colsample_bytree)
                    answers_grid['learning_rate'].append(learning_rate)
                    answers_grid['max_depth'].append(max_depth)
                    answers_grid['alpha'].append(alpha)
                    answers_grid['n_estimators'].append(n_estimators)

                    print(f"combination {index}")
                    print(f"colsample_bytree : {colsample_bytree}, learning_rate : {learning_rate}, max_depth : {max_depth}, alpha : {alpha}, n_estimators : {n_estimators}")
                    print(f"Train_accuracy : {train_accuracy:.2f}")
                    print(f"Test_accuracy : {test_accuracy:.2f}")
                    print("-"*30)

In [ ]:
accuracy_results=pd.DataFrame()

accuracy_results['combination']=answers_grid['combination']
accuracy_results['train_Accuracy']=answers_grid['train_Accuracy']
accuracy_results['test_Accuracy']=answers_grid['test_Accuracy']
accuracy_results['colsample_bytree']=answers_grid['colsample_bytree']
accuracy_results['learning_rate']=answers_grid['learning_rate']
accuracy_results['max_depth']=answers_grid['max_depth']
accuracy_results['alpha']=answers_grid['alpha']
accuracy_results['n_estimators']=answers_grid['n_estimators']

In [ ]:
accuracy_results.to_excel('accuracy_results.xlsx',sheet_name='hyperparamter tuning')

### Unseen Data

In [96]:
a3 = pd.read_excel('Unseen_Dataset.xlsx')

In [97]:
cols_in_df=list(df.columns)
cols_in_df

['pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'Tot_TL_closed_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'CC_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'max_recent_level_of_deliq',
 'num_deliq_6_12mts',
 'num_times_60p_dpd',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_12mts',
 'num_lss',
 'recent_level_of_deliq',
 'CC_enq_L12m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L3m',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag',
 'MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2',
 'Approved_Flag']

### Drop Approved_Flag Column

In [98]:
cols_in_df=cols_in_df[:-1]
cols_in_df

['pct_tl_open_L6M',
 'pct_tl_closed_L6M',
 'Tot_TL_closed_L12M',
 'pct_tl_closed_L12M',
 'Tot_Missed_Pmnt',
 'CC_TL',
 'Home_TL',
 'PL_TL',
 'Secured_TL',
 'Unsecured_TL',
 'Other_TL',
 'Age_Oldest_TL',
 'Age_Newest_TL',
 'time_since_recent_payment',
 'max_recent_level_of_deliq',
 'num_deliq_6_12mts',
 'num_times_60p_dpd',
 'num_std_12mts',
 'num_sub',
 'num_sub_6mts',
 'num_sub_12mts',
 'num_dbt',
 'num_dbt_12mts',
 'num_lss',
 'recent_level_of_deliq',
 'CC_enq_L12m',
 'PL_enq_L12m',
 'time_since_recent_enq',
 'enq_L3m',
 'NETMONTHLYINCOME',
 'Time_With_Curr_Empr',
 'CC_Flag',
 'PL_Flag',
 'pct_PL_enq_L6m_of_ever',
 'pct_CC_enq_L6m_of_ever',
 'HL_Flag',
 'GL_Flag',
 'MARITALSTATUS',
 'EDUCATION',
 'GENDER',
 'last_prod_enq2',
 'first_prod_enq2']

In [99]:
df_unseen = a3[cols_in_df]

df_unseen.shape

(100, 42)

In [100]:
print(df_unseen['MARITALSTATUS'].unique())
print(df_unseen['EDUCATION'].unique())
print(df_unseen['GENDER'].unique())
print(df_unseen['last_prod_enq2'].unique())
print(df_unseen['first_prod_enq2'].unique())

['Married' 'Single']
['12TH' 'GRADUATE' 'SSC' 'POST-GRADUATE' 'UNDER GRADUATE' 'OTHERS']
['M' 'F']
['PL' 'ConsumerLoan' 'AL' 'CC' 'others' 'HL']
['PL' 'ConsumerLoan' 'others' 'AL' 'HL' 'CC']


In [101]:
df_unseen.loc[df_unseen['EDUCATION']=='SSC',['EDUCATION']]=1
df_unseen.loc[df_unseen['EDUCATION']=='12TH',['EDUCATION']]=2
df_unseen.loc[df_unseen['EDUCATION']=='GRADUATE',['EDUCATION']]=3
df_unseen.loc[df_unseen['EDUCATION']=='UNDER GRADUATE',['EDUCATION']]=3
df_unseen.loc[df_unseen['EDUCATION']=='POST-GRADUATE',['EDUCATION']]=4
df_unseen.loc[df_unseen['EDUCATION']=='OTHERS',['EDUCATION']]=1
df_unseen.loc[df_unseen['EDUCATION']=='PROFESSIONAL',['EDUCATION']]=3

In [102]:
df_unseen['EDUCATION'].value_counts()

EDUCATION
3    41
2    28
1    26
4     5
Name: count, dtype: int64

In [103]:
df_unseen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            100 non-null    float64
 1   pct_tl_closed_L6M          100 non-null    float64
 2   Tot_TL_closed_L12M         100 non-null    int64  
 3   pct_tl_closed_L12M         100 non-null    float64
 4   Tot_Missed_Pmnt            100 non-null    int64  
 5   CC_TL                      100 non-null    int64  
 6   Home_TL                    100 non-null    int64  
 7   PL_TL                      100 non-null    int64  
 8   Secured_TL                 100 non-null    int64  
 9   Unsecured_TL               100 non-null    int64  
 10  Other_TL                   100 non-null    int64  
 11  Age_Oldest_TL              100 non-null    int64  
 12  Age_Newest_TL              100 non-null    int64  
 13  time_since_recent_payment  100 non-null    int64  


### convert education column type into integer

In [104]:
df_unseen['EDUCATION']=df_unseen['EDUCATION'].astype('int')
df_unseen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 42 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   pct_tl_open_L6M            100 non-null    float64
 1   pct_tl_closed_L6M          100 non-null    float64
 2   Tot_TL_closed_L12M         100 non-null    int64  
 3   pct_tl_closed_L12M         100 non-null    float64
 4   Tot_Missed_Pmnt            100 non-null    int64  
 5   CC_TL                      100 non-null    int64  
 6   Home_TL                    100 non-null    int64  
 7   PL_TL                      100 non-null    int64  
 8   Secured_TL                 100 non-null    int64  
 9   Unsecured_TL               100 non-null    int64  
 10  Other_TL                   100 non-null    int64  
 11  Age_Oldest_TL              100 non-null    int64  
 12  Age_Newest_TL              100 non-null    int64  
 13  time_since_recent_payment  100 non-null    int64  


In [105]:
df_encoded_unseen=pd.get_dummies(df_unseen,columns=['MARITALSTATUS','GENDER', 'last_prod_enq2', 'first_prod_enq2'],dtype='int')

In [106]:
df_encoded_unseen.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 54 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   pct_tl_open_L6M               100 non-null    float64
 1   pct_tl_closed_L6M             100 non-null    float64
 2   Tot_TL_closed_L12M            100 non-null    int64  
 3   pct_tl_closed_L12M            100 non-null    float64
 4   Tot_Missed_Pmnt               100 non-null    int64  
 5   CC_TL                         100 non-null    int64  
 6   Home_TL                       100 non-null    int64  
 7   PL_TL                         100 non-null    int64  
 8   Secured_TL                    100 non-null    int64  
 9   Unsecured_TL                  100 non-null    int64  
 10  Other_TL                      100 non-null    int64  
 11  Age_Oldest_TL                 100 non-null    int64  
 12  Age_Newest_TL                 100 non-null    int64  
 13  time_s

In [107]:
k=df_encoded.describe()
k

,pct_tl_open_L6M,pct_tl_closed_L6M,Tot_TL_closed_L12M,pct_tl_closed_L12M,Tot_Missed_Pmnt,CC_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL,time_since_recent_payment,max_recent_level_of_deliq,num_deliq_6_12mts,num_times_60p_dpd,num_std_12mts,num_sub,num_sub_6mts,num_sub_12mts,num_dbt,num_dbt_12mts,num_lss,recent_level_of_deliq,CC_enq_L12m,PL_enq_L12m,time_since_recent_enq,enq_L3m,NETMONTHLYINCOME,Time_With_Curr_Empr,CC_Flag,PL_Flag,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,EDUCATION,MARITALSTATUS_Married,MARITALSTATUS_Single,GENDER_F,GENDER_M,last_prod_enq2_AL,last_prod_enq2_CC,last_prod_enq2_ConsumerLoan,last_prod_enq2_HL,last_prod_enq2_PL,last_prod_enq2_others,first_prod_enq2_AL,first_prod_enq2_CC,first_prod_enq2_ConsumerLoan,first_prod_enq2_HL,first_prod_enq2_PL,first_prod_enq2_others
count,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,4.206200e+04,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000,42062.000000
mean,0.179037,0.097784,0.825496,0.160356,0.525748,0.145927,0.076245,0.328016,2.921354,2.341686,1.116518,46.498407,13.970353,218.608863,14.315439,0.336979,0.438900,3.280134,0.063834,0.002211,0.009224,0.024511,0.004279,0.016713,11.804479,0.268936,0.779207,264.858946,1.230446,2.692989e+04,110.348224,0.102967,0.193072,0.195506,0.064189,0.252223,0.056583,2.313704,0.734273,0.265727,0.112192,0.887808,0.032167,0.052185,0.391779,0.019733,0.179568,0.324568,0.062788,0.047264,0.263302,0.030645,0.105344,0.490657
std,0.278048,0.210962,1.537244,0.258832,1.106463,0.549326,0.358590,0.916387,6.379907,3.405471,2.486854,42.110115,18.835571,422.291127,54.057498,1.097380,2.148449,7.566458,0.800008,0.081706,0.220791,0.621904,0.184465,0.573775,46.423123,1.019482,1.802131,466.594481,2.069484,2.084330e+04,75.630925,0.303919,0.394714,0.367420,0.225993,0.434294,0.231047,0.871061,0.441725,0.441725,0.315606,0.315606,0.176445,0.222402,0.488153,0.139082,0.383832,0.468219,0.242585,0.212205,0.440430,0.172357,0.307000,0.499919
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,14.000000,4.000000,51.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,1.800000e+04,61.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,34.000000,7.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,79.000000,1.000000,2.400000e+04,92.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,

### Model Using Hyperparameter Values

In [109]:
model=XGBClassifier(objective='multi:softmax',
                    num_class=4,
                    colsample_bytree=0.9,
                    learning_rate=1,
                    max_depth=3,
                    alpha=10,
                    n_estimators=100)


model.fit(X_train,y_train)

XGBClassifier(alpha=10, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=4, ...)

In [111]:
y_pred_unseen=model.predict(df_encoded_unseen)

In [116]:
a3['Target Variable']=y_pred_unseen

In [117]:
a3.to_excel('final_prediction.xlsx',index=False)